<a href="https://colab.research.google.com/github/anshupandey/Working_with_Large_Language_models/blob/main/WWL_C13A_Deploying_LLM_Application_FastAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Product Description as a Web Service

* Web Service Framework: FastAPI
* LLM: Gemini 1.5 Flasj
* Tunneling Tool: NGrok

In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn google-cloud-aiplatform --quiet

In [2]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [3]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import vertexai
from vertexai.generative_models import GenerationConfig, GenerativeModel

PROJECT_ID = "jrproject-402905"
LOCATION = "us-central1"
MODEL_ID = "gemini-1.5-flash-preview-0514"
vertexai.init(project=PROJECT_ID, location=LOCATION)
model = GenerativeModel(MODEL_ID)
generation_config = GenerationConfig( temperature=0.9, top_k=32,)

# Initialize FastAPI client
app = FastAPI()

# Create class with pydantic BaseModel
class SampleRequest(BaseModel):
    product_name: str
    product_brief: str

def get_product_description(product_name,product_brief):
  prompt = f""" For the given below product details, provide a 2 line product description.
  Do not add any additional unknown information, just provide the product description.
  Product Name: {product_name}
  Product Brief: {product_brief} """
  response = model.generate_content([prompt], generation_config=generation_config,)
  return response.text

@app.post("/trial/")  # This line decorates 'get_product_description' function as a POST endpoint
async def generate(request: SampleRequest):
    try:
        product_desc = get_product_description(request.product_name,request.product_brief)
        return {"product_description": product_desc}
    except Exception as e:
        # Handle exceptions or errors during text generation
        raise HTTPException(status_code=500, detail=str(e))

# Setting up NGRok
https://dashboard.ngrok.com/signup

In [4]:
!ngrok config add-authtoken 2id4ZvjA88kTS4ZDegoD2sRkirR_61gPMN1rcHWkEsajB7c1Z

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [2231]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://759b-34-44-25-163.ngrok-free.app
INFO:     35.230.75.87:0 - "POST /trial HTTP/1.1" 307 Temporary Redirect
INFO:     35.230.75.87:0 - "POST /trial/ HTTP/1.1" 200 OK
